In [6]:
import pandas as pd
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense,Dropout,Activation,Conv1D,MaxPooling1D,Flatten,MaxPool1D,BatchNormalization,LeakyReLU
from keras.callbacks import ReduceLROnPlateau,TensorBoard
from keras.utils import to_categorical
from keras.regularizers import l2
from keras.optimizers import Adamax,Adagrad
import numpy as np
from keras.models import load_model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [2]:
feature_names = ['GR', 'ILD_log10', 'DeltaPHI', 'PHIND', 'PE', 'NM_M', 'RELPOS']
facies_names = ['SS', 'CSiS', 'FSiS', 'SiSh', 'MS', 'WS', 'D', 'PS', 'BS']
facies_colors =  ['#F56F45', '#FA7E7E','#8C8456','#8F6A9E',
       '#226854','#9EDAAB', '#7C97FC', '#25C4EF', '#A7D909']

In [4]:
data = pd.read_csv('facies_data.csv')
def one_hot_enc(y):
    classes = sorted(list(set(y)))
    y_enc = np.zeros((y.shape[0],len(classes)),dtype="float32")
    for index,label in enumerate(y):
        y_enc[index,classes.index(label)] = 1.
    return y_enc

In [ ]:
X = data[feature_names].values  # features
print(X.shape)
y = data['Facies'].values  # labels
well = data['Well Name'].values

depth = data['Depth'].values
#nan_idx = np.any(np.isnan(X), axis=1)
#X = X[np.logical_not(nan_idx), :]


imp = preprocessing.Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(X)
X = imp.transform(X)

max_values_x = np.amax(X,axis=0)#normalization
X/=np.amax(X,axis=0)#normalization

print (X.shape)
print (y[50])
#y = y[np.logical_not(nan_idx)]
classes = set(y)
print (classes)
y = one_hot_enc(y)
print (y[50])


In [ ]:
# Simple NN
units = 256
model = Sequential([
    
    Dense(units,input_shape=(7,)),
    Activation("relu"),
    Dropout(0.3),
    Dense(units),
    Activation("relu"),
    Dropout(0.3),
    Dense(len(classes)),
    Activation("softmax"),
    
])
model.compile(loss='categorical_crossentropy',optimizer="adagrad",metrics=['accuracy'])

In [ ]:
model.fit(X,y,validation_split=0.2,batch_size=10,epochs=400,callbacks=[checkpoint,tbCallBack],shuffle=True)

In [ ]:
units = 256
model = Sequential([
    
    Dense(units,input_shape=(7,)),
    Activation("relu"),
    Dropout(0.3),
    Dense(units),
    Activation("relu"),
    Dropout(0.3),
    Dense(len(classes)),
    Activation("softmax"),
    
])
model.compile(loss='categorical_crossentropy',optimizer="adagrad",metrics=['accuracy'])


In [ ]:
model.fit(X,y,validation_split=0.2,batch_size=10,epochs=400,callbacks=[checkpoint,tbCallBack],shuffle=True)

In [ ]:
%matplotlib inline
history = cnn_model.fit(X,y,validation_split=0.2,batch_size = 10,epochs=500,shuffle=True)

In [ ]:
plt.figure(figsize=(12,12))
plt.subplot()
plt.plot(history.history['loss'],color='magenta')
plt.plot(history.history['val_loss'],color='darkcyan')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'],loc='best')
plt.show()
plt.figure(figsize=(12,12))
plt.subplot()
plt.plot(history.history['acc'],color='turquoise')
plt.plot(history.history['val_acc'],color='mediumslateblue')
plt.title('model accuracy')
plt.xlabel('epoch')
plt.ylabel('acc')
plt.legend(['train','test'],loc='best')
plt.show()

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

HERE 1D-CNN

In [ ]:
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)
model = Sequential([
    
    Conv1D(64, 2, strides=1,input_shape =(7,1)), 
    Activation("relu"), 
    BatchNormalization(),
    #MaxPool1D(2,1),
   

    Conv1D(64, 2, strides=1,padding="same"), 
    Activation("relu"), 
    BatchNormalization(),
    #MaxPool1D(2,1),
    Dropout(0.3),
    
    Conv1D(64,2,strides=1,padding="same"),
    Activation("relu"), 
    BatchNormalization(),
    MaxPooling1D(3,1),
   
   
    Conv1D(128,2,strides=1,padding="same"),
    Activation("relu"), 
    BatchNormalization(),
    #MaxPooling1D(2,1),
    
    
    Conv1D(128,2,strides=1,padding="same"),
    Activation("relu"), 
    BatchNormalization(),
    Dropout(0.3),
    
    Conv1D(128,1,strides=1,padding="same"),
    Activation("relu"), 
    BatchNormalization(),
    MaxPooling1D(1,1),
    
    
    Flatten(), 
    Dense(2500),
    Activation("relu"),
    BatchNormalization(),
   
    Dropout(0.3),
    Dense(1500),
    Activation("relu"),
    
    Dense(len(classes)),
    Activation("softmax"),
    
])
model.summary()
model.compile(loss='categorical_crossentropy',optimizer="adagrad",metrics=['accuracy'])

In [ ]:
#model = load_model("weights/weights-488-0.74.h5")
filepath = "weights/weights_NOT_ok_-{epoch:02d}-{val_acc:.2f}.h5"
tbCallBack = TensorBoard(log_dir = 'Graph/CNN13_graph_not_ok', histogram_freq=0, write_graph=True, write_images=True)
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max',
                                     save_weights_only=False)
model.fit(X_train,y_train,shuffle=True,validation_data = (X_test, y_test),batch_size=64,epochs=2800,callbacks=[checkpoint,tbCallBack])

In [ ]:
# HERE HERE HERE

In [13]:
import pickle
with open('train_not_ok.pickle', 'rb') as f:
    X_train,y_train, X_test, y_test = pickle.load(f)

In [14]:
model2 = load_model('C:/w_oknotok/weights_NOT_ok_-1403-0.75.h5')

In [15]:
#X_test = X_test.reshape((830, 7,1))
pred = model2.predict_classes(X_test)
y_test_dec = np.argmax(y_test,axis=1)
accurc = accuracy_score(y_test_dec,pred)
f1_ = f1_score(y_test_dec,pred,average='macro')

In [16]:
accurc
# 0.7481927710843373

0.7481927710843373

In [17]:
f1_
#0.7520486563765958

0.7520486563765958

In [7]:
adjacent_facies = np.array([[1,2], [1,2,3], [2,3], [4,5], [4,5,6], [5,6,7], [6,7,8], [6,7,8,9], [7,8,9]])
p = 0

for i in range(1, len(pred)):
    if (pred[i] == adjacent_facies[pred[i]-1]).any():
        p = p + 1
accuracy_adj = p/float(len(pred))
print('Adjacent facies classification accuracy = %f' % accuracy_adj)

In [23]:
Precision = {}
Recall = {}
y_test1_ = list(y_test)
for j in range(1, 10):
    TP,FP,FN,TN = 0,0,0,0
    for i in range(1, len(pred)):
        if ((y_test_dec[i] == j) and (pred[i] == j)):
            TP += 1
        if ((y_test_dec[i] != j) and (pred[i] == j)):
            FP += 1
        if ((y_test_dec[i] == j) and (pred[i] != j)):
            FN += 1
        if ((y_test_dec[i] != j) and (pred[i] != j)):
            TN += 1   
    if (TP+FP == 0):
        Precision[j] = 0
    else:
        Precision[j] = float(TP)/(TP+FP)
    if (TP+FN == 0):
        Recall[j] = 0
    else:
        Recall[j] = float(TP)/(TP+FN)

In [22]:
y_test

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [ ]:
# y_test_dec,pred
# Считаем по классам

In [ ]:
filepath = "C:\eto_CNN\weights-{epoch:02d}-{val_acc:.2f}.h5"
tbCallBack = TensorBoard(log_dir = 'C:\eto_CNN_graph', histogram_freq=0, write_graph=True, write_images=True)
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max',
                                     save_weights_only=False)

model.fit(X,y,validation_split=0.2,batch_size=10,epochs=500,callbacks=[checkpoint,tbCallBack],shuffle=True)

In [ ]:
filepath = "C:\eto_CNN2\weights-{epoch:02d}-{val_acc:.2f}.h5"
tbCallBack = TensorBoard(log_dir = 'C:\eto_CNN_graph2', histogram_freq=0, write_graph=True, write_images=True)
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max',
                                     save_weights_only=False)

model.fit(X,y,validation_split=0.2,batch_size=10,epochs=500,callbacks=[checkpoint,tbCallBack],shuffle=True)

In [ ]:
###############################################################################################################################

In [ ]:
X = np.expand_dims(X, axis=2)
model = Sequential([
    
    Conv1D(32, 4, strides=1,input_shape =(7,1),padding="same"), 
    Activation("relu"), 
   
    Dropout(0.3),
   
    Conv1D(64, 4, strides=1,padding="same"),
    Activation("relu"),
    
    Dropout(0.3),
    
    Conv1D(128, 4, strides=1,padding="same"), 
    Activation("relu"), 
    
   
    Dropout(0.3),
    
    Flatten(), 
    Dense(256),
    Activation("relu"),
   
    Dropout(0.3),
    
    Dense(len(classes)),
    Activation("softmax"),
    
])
model.summary()
model.compile(loss='categorical_crossentropy',optimizer="rmsprop",metrics=['accuracy'])

In [ ]:
filepath = "C:\cnn_1d\weights-{epoch:02d}-{val_acc:.2f}.h5"
tbCallBack = TensorBoard(log_dir = 'C:\cnn1d_graph', histogram_freq=0, write_graph=True, write_images=True)
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max',
                                     save_weights_only=False)

model.fit(X,y,validation_split=0.2,batch_size=10,epochs=4000,callbacks=[checkpoint,tbCallBack],shuffle=True)

In [ ]:
import keras
keras.callbacks.History